In [23]:
# Import Dependencies
import matplotlib.pyplot as plt
import requests as req
import pandas as pd
import numpy as np
import seaborn
import random
import datetime
import time
import json
import os

In [2]:
csv_path = os.path.join('marvel_data.csv')

marvel_df = pd.read_csv(csv_path)

In [110]:
marvel_df['name'][:2500].head()

0         Spider-Man 
1    Captain America 
2          Wolverine 
3           Iron Man 
4               Thor 
Name: name, dtype: object

In [109]:
loop_test = ['Iron Man', 'Captain America', 'Thor', 'Black Widow', 'Hulk']

In [7]:
api_key = '30560b9f10847a0a56548235e91450ea'

## API Request and Data Retrieval

In [81]:
import hashlib
import urllib.parse
import urllib.request

In [83]:
url = 'https://gateway.marvel.com:443/v1/public/characters'

apikey = '30560b9f10847a0a56548235e91450ea'
privateKey = '2ed1b2554627b2e37a3adfc916399a58d8fa350a'

ts = str(round(time.time()))

hsh = hashlib.md5(bytes(ts+privateKey+apikey, 'utf-8')).hexdigest()

In [131]:
url = 'https://gateway.marvel.com:443/v1/public/characters'

apikey = '30560b9f10847a0a56548235e91450ea'
private_key = '2ed1b2554627b2e37a3adfc916399a58d8fa350a'



#Initialize python dict containing data

marvel_data = []


for hero in marvel_df['name'][:5]:
    
    search_data = {'id':[],
                'name':[],
               'description':[],
                'comics_avail':[],
                'comics_list':[],
                'series_avail':[],
                'series_list':[],
                'events_avail':[],
                'events_list':[]}
    
    #Set parameters for search
    params = {'ts': ts,
            'apikey': '30560b9f10847a0a56548235e91450ea',
              'hash': hsh,
             'name': hero,
             'limit': '100'}
    
    #Set up response request
    response = req.get(url, params=params).json()
    hero_data = response['data']['results'][0]
    
    try:

        #Extract data initial data
        search_data['id'].append(hero_data['id'])
        search_data['name'].append(hero_data['name'])
        search_data['description'].append(hero_data['description'])
        search_data['comics_avail'].append(hero_data['comics']['available'])
        search_data['series_avail'].append(hero_data['series']['available'])
        search_data['events_avail'].append(hero_data['events']['available'])

        #Set length of comics, series, and events to iterate over
        comics_length = len(hero_data['comics']['items'])
        series_length = len(hero_data['series']['items'])
        events_length = len(hero_data['events']['items'])

        #Iterate through items of comics, series and events to obtain names
        for n in range(comics_length):
            comics_list = hero_data['comics']['items'][n]
            search_data['comics_list'].append(comics_list['name'])

        for n in range(series_length):
            series_list = hero_data['series']['items'][n]
            search_data['series_list'].append(series_list['name'])

        for n in range(events_length):
            events_list = hero_data['series']['items'][n]
            search_data['events_list'].append(events_list['name'])
            
        marvel_data.append(search_data.copy())
        
    except IndexError:
        print('Missing or Invalid')
        print(f'Having Error with {hero}')

5

## Adapted from brandonrose.org

In [114]:
import numpy as np
import pandas as pd
import nltk
nltk.download('stopwords')
import re
import os
import codecs
from sklearn import feature_extraction

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mgarcia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [115]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [117]:
print(stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [118]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [119]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [134]:
marvel_data

[{'comics_avail': [3174],
  'comics_list': ['Spider-Man: 101 Ways to End the Clone Saga (1997) #1',
   'A Year of Marvels (Trade Paperback)',
   'A Year of Marvels: April Infinite Comic (2016) #1',
   'A+X (2012) #4',
   'Actor Presents Spider-Man and the Incredible Hulk (2003) #1',
   'Age of Heroes (2010) #1',
   'Age of Heroes (Trade Paperback)',
   'Alias Omnibus (Hardcover)',
   'Alpha: Big Time (2013) #1',
   'Alpha Flight (1983) #1',
   'Alpha Flight (1983) #127',
   'Alpha Flight Classic Vol. 1 (Trade Paperback)',
   'Alpha: Big Time (2013) #0.1',
   'Amazing Fantasy (1962) #15',
   'Amazing Fantasy (2004) #15',
   'Amazing Fantasy 15: Spider-Man! (2011) #1',
   'AMAZING FANTASY OMNIBUS HC (Hardcover)',
   'Amazing Spider-Girl (2006)',
   'Amazing Spider-Girl (2006) #11',
   'AMAZING SPIDER-GIRL VOL. 1: WHATEVER HAPPENED TO THE DAUGHTER OF SPIDER-MAN TPB (Trade Paperback)'],
  'description': ['Bitten by a radioactive spider, high school student Peter Parker gained the speed, st

In [136]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mgarcia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [151]:
print(type(str(marvel_data[0]['description']).strip('[]')))

<class 'str'>


In [167]:
names = []
desc = []

for n in range(len(marvel_data)):
    names.append(marvel_data[n]['name'][0])
    desc.append(marvel_data[n]['description'][0])

In [169]:
#not super pythonic, no, not at all.
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in desc:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [193]:
len(totalvocab_stemmed)==len(totalvocab_tokenized)

True

In [194]:
totalvocab_stemmed

['bitten',
 'by',
 'a',
 'radioact',
 'spider',
 'high',
 'school',
 'student',
 'peter',
 'parker',
 'gain',
 'the',
 'speed',
 'strength',
 'and',
 'power',
 'of',
 'a',
 'spider',
 'adopt',
 'the',
 'name',
 'spider-man',
 'peter',
 'hope',
 'to',
 'start',
 'a',
 'career',
 'use',
 'his',
 'new',
 'abil',
 'taught',
 'that',
 'with',
 'great',
 'power',
 'come',
 'great',
 'respons',
 'spidey',
 'has',
 'vow',
 'to',
 'use',
 'his',
 'power',
 'to',
 'help',
 'peopl',
 'vow',
 'to',
 'serv',
 'his',
 'countri',
 'ani',
 'way',
 'he',
 'could',
 'young',
 'steve',
 'roger',
 'took',
 'the',
 'super',
 'soldier',
 'serum',
 'to',
 'becom',
 'america',
 "'s",
 'one-man',
 'armi',
 'fight',
 'for',
 'the',
 'red',
 'white',
 'and',
 'blue',
 'for',
 'over',
 'year',
 'captain',
 'america',
 'is',
 'the',
 'live',
 'breath',
 'symbol',
 'of',
 'freedom',
 'and',
 'liberti',
 'born',
 'with',
 'super-human',
 'sens',
 'and',
 'the',
 'power',
 'to',
 'heal',
 'from',
 'almost',
 'ani',
 

In [192]:
totalvocab_tokenized

['bitten',
 'by',
 'a',
 'radioactive',
 'spider',
 'high',
 'school',
 'student',
 'peter',
 'parker',
 'gained',
 'the',
 'speed',
 'strength',
 'and',
 'powers',
 'of',
 'a',
 'spider',
 'adopting',
 'the',
 'name',
 'spider-man',
 'peter',
 'hoped',
 'to',
 'start',
 'a',
 'career',
 'using',
 'his',
 'new',
 'abilities',
 'taught',
 'that',
 'with',
 'great',
 'power',
 'comes',
 'great',
 'responsibility',
 'spidey',
 'has',
 'vowed',
 'to',
 'use',
 'his',
 'powers',
 'to',
 'help',
 'people',
 'vowing',
 'to',
 'serve',
 'his',
 'country',
 'any',
 'way',
 'he',
 'could',
 'young',
 'steve',
 'rogers',
 'took',
 'the',
 'super',
 'soldier',
 'serum',
 'to',
 'become',
 'america',
 "'s",
 'one-man',
 'army',
 'fighting',
 'for',
 'the',
 'red',
 'white',
 'and',
 'blue',
 'for',
 'over',
 'years',
 'captain',
 'america',
 'is',
 'the',
 'living',
 'breathing',
 'symbol',
 'of',
 'freedom',
 'and',
 'liberty',
 'born',
 'with',
 'super-human',
 'senses',
 'and',
 'the',
 'power',

In [170]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 252 items in vocab_frame


In [171]:
print(vocab_frame.head())
print
print
print
print

                words
bitten         bitten
by                 by
a                   a
radioact  radioactive
spider         spider


<function print>

In [190]:
desc

['Bitten by a radioactive spider, high school student Peter Parker gained the speed, strength and powers of a spider. Adopting the name Spider-Man, Peter hoped to start a career using his new abilities. Taught that with great power comes great responsibility, Spidey has vowed to use his powers to help people.',
 "Vowing to serve his country any way he could, young Steve Rogers took the super soldier serum to become America's one-man army. Fighting for the red, white and blue for over 60 years, Captain America is the living, breathing symbol of freedom and liberty.",
 "Born with super-human senses and the power to heal from almost any wound, Wolverine was captured by a secret Canadian organization and given an unbreakable skeleton and claws. Treated like an animal, it took years for him to control himself. Now, he's a premiere member of both the X-Men and the Avengers.",
 'Wounded, captured and forced to build a weapon by his enemies, billionaire industrialist Tony Stark instead created

In [173]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(desc) #fit the vectorizer to descriptions

print(tfidf_matrix.shape)

CPU times: user 12 ms, sys: 4.37 ms, total: 16.3 ms
Wall time: 21.4 ms
(5, 408)


In [174]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print
print

<function print>

In [186]:
from sklearn.cluster import KMeans

num_clusters = 3

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

CPU times: user 31.7 ms, sys: 3.97 ms, total: 35.7 ms
Wall time: 32.1 ms


In [187]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [188]:
films = { 'hero': names, 'description': desc, 'cluster': clusters} #Could include comics, series, events, stories available

frame = pd.DataFrame(films, index = [clusters] , columns = ['hero', 'description', 'cluster'])

In [189]:
frame

,hero,description,cluster
1,Spider-Man,"Bitten by a radioactive spider, high school st...",1
0,Captain America,"Vowing to serve his country any way he could, ...",0
0,Wolverine,Born with super-human senses and the power to ...,0
1,Iron Man,"Wounded, captured and forced to build a weapon...",1
2,Thor,"As the Norse God of thunder and lightning, Tho...",2
